# TI3145TU Midterm Assignment 
## Football Players Wages

We hope you enjoy this assignment, good luck!

Student names: XXX

Student numbers: XXX

### Imports

In [4]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor

import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

### Load data

aaaa

In [9]:
# These are your training samples along with their labels
data = pd.read_csv('football_wages.csv')
data.head()

# You need to extract the features and the regression target. The regression target is 'log_wages'.

# sort the data in ascending order of average log_wages by nationality
mean_by_nationality = data.groupby('nationality_name')['log_wages'].mean().reset_index()
mean_by_nationality_sorted = mean_by_nationality.sort_values(by='log_wages', ascending=False)
sorted_data = pd.merge(data, mean_by_nationality_sorted, on='nationality_name', suffixes=('', '_mean'))
sorted_data = sorted_data.sort_values(by='log_wages_mean', ascending=False).drop(columns='log_wages_mean')

# convert the nationality strings to integers, with 0 being the country with lowest average log_wages
sorted_data['nationality_name'], uniques = pd.factorize(data['nationality_name'])

X = sorted_data.iloc[:, :28]
y = sorted_data.iloc[:, 28]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Autograder 

In the autograder you will need to provide two things: 1) estimate of the MAE of your model on unseen data, 2) the predictions on the autograder data. For the autograder data we only provide the features and not the regression targets. Thus, you cannot compute the MAE on this data yourself - you need to estimate that with the data provided above. 

In [6]:
data_autograder = pd.read_csv('football_autograder.csv')
data_autograder.head()


In [7]:
ks = [1, 2, 5, 10, 20, 30, 50, 100]

err_test = []

for s in ks:
    knc = KNeighborsRegressor(n_neighbors=s)

    knc.fit(X_train, y_train)

    test_predictions = knc.predict(X_test)
    train_predictions = knc.predict(X_train)
    
    err_test.append(mean_absolute_error(y_test, test_predictions))

# END ANSWER

best_k_index = np.argmin(err_test)
best_k = ks[best_k_index]
print("The best k on the test set is %d" % best_k)
print("The lowest error is ", err_test[best_k_index])
print(10 ** err_test[best_k_index])


In [22]:
# Instantiate the SGDRegressor
sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)

# Define the parameter grid for alpha (regularization strength) and learning rate
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Tuning regularization strength
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],  # Different learning rate strategies
    'eta0': [0.001, 0.01, 0.1, 1]  # Initial learning rate for strategies that require it
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(sgd_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_scaled, y_train)

# Get the best model from grid search
best_sgd = grid_search.best_estimator_

# Predict on the test set with the best model
y_pred = best_sgd.predict(X_test_scaled)

# Evaluate the model using MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Best MAE: {mae}")
print(f"Best Parameters: {grid_search.best_params_}")

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stocha

Best MAE: 0.27692806009280757
Best Parameters: {'alpha': 0.0001, 'eta0': 0.1, 'learning_rate': 'adaptive'}


In [ ]:
# TODO Replace this with your own estimate of the MAE of your best model
estimate_MAE_on_new_data = np.array([1.0])

# TODO Replace this with the predictions of your best model
# via e.g. prediction = model.predict(data_autograder)
# your predictions here should again be the $log_{10}(wage)$ of the football player, just as in the provided data. 
predictions_autograder_data = np.array([-1] * 14178)

# Upload this file to the Vocareum autograder:
result = np.append(estimate_MAE_on_new_data, predictions_autograder_data)
pd.DataFrame(result).to_csv("autograder_submission.txt", index=False, header=False)